In [ ]:
import pyaudiowpatch as pyaudio
import threading
import time
import wave

# Initialize PyAudio
pa = pyaudio.PyAudio()

# Function to record from a specific device
def record_from_device(device_index, duration=10, output_filename="output.wav"):
    stream = None
    try:
        mic = pa.get_device_info_by_index(device_index)
        mic_rates = mic['defaultSampleRate']
        mic_channels = mic['maxInputChannels']
        is_loopback = mic.get('isLoopbackDevice', False)
        
        # For loopback devices, use the actual channel count; for mics, use 1
        if is_loopback:
            channels_to_use = int(mic_channels)
        else:
            channels_to_use = int(min(int(mic_channels), 1)) if int(mic_channels) > 0 else 1
            
        print(f"Device {device_index}: {mic['name']}")
        print(f"  Channels: {channels_to_use}, Rate: {int(mic_rates)}, Loopback: {is_loopback}")

        chunk_size = 2048 
        stream = pa.open(format=pyaudio.paInt16,
                         channels=channels_to_use,
                         rate=int(mic_rates),
                         input=True,
                         input_device_index=device_index,
                         frames_per_buffer=chunk_size)
        
        print(f"Recording from device {device_index}...")

        frames = []
        start_time = time.time()
        
        if is_loopback:
            # For loopback devices, use time-based recording with timeout
            while time.time() - start_time < duration:
                try:
                    # Check if data is available
                    available = stream.get_read_available()
                    if available >= chunk_size:
                        data = stream.read(chunk_size, exception_on_overflow=False)
                        frames.append(data)
                    elif available > 0:
                        data = stream.read(available, exception_on_overflow=False)
                        frames.append(data)
                    else:
                        # Sleep briefly to prevent busy waiting
                        time.sleep(0.01)
                        
                except OSError as e:
                    print(f"Loopback error on device {device_index}: {e}")
                    time.sleep(0.01)
                    continue
        else:
            # For microphone devices, use chunk-based recording
            total_chunks = int(mic_rates / chunk_size * duration)
            for _ in range(total_chunks):
                try:
                    data = stream.read(chunk_size, exception_on_overflow=False)
                    frames.append(data)
                except OSError as e:
                    print(f"Mic error on device {device_index}: {e}")
                    continue

        # Write to WAV file
        with wave.open(output_filename, 'wb') as wf:
            wf.setnchannels(channels_to_use)
            wf.setsampwidth(pa.get_sample_size(pyaudio.paInt16))
            wf.setframerate(int(mic_rates))
            if frames:
                wf.writeframes(b''.join(frames))

        print(f"Finished recording from device {device_index}. Frames: {len(frames)}")
        
    except Exception as e:
        print(f"Error recording from device {device_index}: {str(e)}")
    finally:
        if stream is not None:
            try:
                stream.stop_stream()
                stream.close()
            except Exception as e:
                print(f"Error closing stream for device {device_index}: {str(e)}")

# Create and start threads for each device with a small delay between starts
device_threads = []
for i, device_index in enumerate([9, 10]):  # Device 9 and Device 10
    output_filename = f"device_{device_index}_output.wav"
    thread = threading.Thread(target=record_from_device, args=(device_index, 10, output_filename))
    device_threads.append(thread)
    thread.start()
    
    # Small delay between starting devices to reduce initial buffer conflicts
    if i < len([9, 10]) - 1:
        time.sleep(0.1)

# Wait for all threads to finish
for thread in device_threads:
    thread.join()

# Terminate PyAudio after recording is finished
pa.terminate()
print("All recordings completed!")

In [ ]:
import pyaudiowpatch as pyaudio
import threading
import time
import wave

class AudioRecorder:
    def __init__(self):
        self.pa = pyaudio.PyAudio()
        self.is_recording = False
        self.device_threads = []
        
    def record_from_device(self, device_index, output_filename="output.wav"):
        stream = None
        try:
            mic = self.pa.get_device_info_by_index(device_index)
            mic_rates = mic['defaultSampleRate']
            mic_channels = mic['maxInputChannels']
            is_loopback = mic.get('isLoopbackDevice', False)
            
            channels_to_use = int(min(int(mic_channels), 1)) if int(mic_channels) > 0 else 1
            print(f"Device {device_index}: {mic['name']}")
            print(f"  Channels: {channels_to_use}, Rate: {int(mic_rates)}, Loopback: {is_loopback}")

            chunk_size = 4096
            stream = self.pa.open(format=pyaudio.paInt16,
                             channels=channels_to_use,
                             rate=int(mic_rates),
                             input=True,
                             input_device_index=device_index,
                             frames_per_buffer=chunk_size)
            
            print(f"Recording from device {device_index}...")

            frames = []
            
            with wave.open(output_filename, 'wb') as wf:
                wf.setnchannels(channels_to_use)
                wf.setsampwidth(self.pa.get_sample_size(pyaudio.paInt16))
                wf.setframerate(int(mic_rates))

                while self.is_recording:
                    try:
                        available = stream.get_read_available()
                        
                        if available >= chunk_size:
                            data = stream.read(chunk_size, exception_on_overflow=False)
                            frames.append(data)
                        elif available > 0:
                            data = stream.read(available, exception_on_overflow=False)
                            frames.append(data)
                        else:
                            time.sleep(0.01)
                        
                        if len(frames) > 100:
                            wf.writeframes(b''.join(frames))
                            frames = []
                            
                    except OSError as e:
                        if "Input overflowed" in str(e):
                            print(f"Buffer overflow on device {device_index}, continuing...")
                            continue
                        else:
                            raise e
                            
                if frames:
                    wf.writeframes(b''.join(frames))

            print(f"Finished recording from device {device_index}.")
            
        except Exception as e:
            print(f"Error recording from device {device_index}: {str(e)}")
        finally:
            if stream is not None:
                try:
                    stream.stop_stream()
                    stream.close()
                except Exception as e:
                    print(f"Error closing stream for device {device_index}: {str(e)}")

    def start_recording(self, device_indices=[9, 10]):
        """Start recording from specified devices"""
        if self.is_recording:
            print("Already recording!")
            return
            
        self.is_recording = True
        self.device_threads = []
        
        for i, device_index in enumerate(device_indices):
            output_filename = f"device_{device_index}_output.wav"
            thread = threading.Thread(
                target=self.record_from_device, 
                args=(device_index, output_filename)
            )
            self.device_threads.append(thread)
            thread.start()
            
            if i < len(device_indices) - 1:
                time.sleep(0.1)
        
        print("Recording started!")
    
    def stop_recording(self):
        """Stop all recording threads"""
        if not self.is_recording:
            print("Not currently recording!")
            return
            
        print("Stopping recording...")
        self.is_recording = False
        
        # Wait for all threads to finish
        for thread in self.device_threads:
            thread.join()
        
        print("All recordings stopped!")
    
    def cleanup(self):
        """Clean up PyAudio resources"""
        self.pa.terminate()

# Example usage (for testing without Streamlit)
if __name__ == "__main__":
    recorder = AudioRecorder()
    
    # Start recording
    recorder.start_recording([9, 10])
    
    # Record for 10 seconds (or wait for external stop signal)
    time.sleep(10)
    
    # Stop recording
    recorder.stop_recording()
    
    # Cleanup
    recorder.cleanup()

In [ ]:
import streamlit as st
from audio_recorder import AudioRecorder 
import time

# Initialize the recorder in session state
if 'recorder' not in st.session_state:
    st.session_state.recorder = AudioRecorder()
    st.session_state.recording = False
    st.session_state.start_time = None

st.title("Audio Recorder Control")

# Device selection
col1, col2 = st.columns(2)
with col1:
    device1 = st.number_input("Device 1 Index", value=9, min_value=0)
with col2:
    device2 = st.number_input("Device 2 Index", value=10, min_value=0)

# Timer fragment - updates automatically without rerunning the whole app
@st.fragment(run_every=0.1)
def timer_display():
    if st.session_state.recording and st.session_state.start_time:
        elapsed = time.time() - st.session_state.start_time
        minutes = int(elapsed // 60)
        seconds = int(elapsed % 60)
        milliseconds = int((elapsed % 1) * 100)
        
        st.markdown(f"### ⏱️ Recording Time: {minutes:02d}:{seconds:02d}.{milliseconds:02d}")
    else:
        st.markdown("### ⏱️ Recording Time: 00:00.00")

# Display the timer
timer_display()

# Control buttons
st.markdown("### Recording Controls")
col1, col2 = st.columns(2)

with col1:
    start_button = st.button(
        "🎤 Start Recording", 
        disabled=st.session_state.recording,
        use_container_width=True,
        type="primary" if not st.session_state.recording else "secondary"
    )
    if start_button:
        st.session_state.recorder.start_recording([int(device1), int(device2)])
        st.session_state.recording = True
        st.session_state.start_time = time.time()
        st.success("Recording started!")
        st.rerun()

with col2:
    stop_button = st.button(
        "⏹️ Stop Recording", 
        disabled=not st.session_state.recording,
        use_container_width=True,
        type="primary" if st.session_state.recording else "secondary"
    )
    if stop_button:
        st.session_state.recorder.stop_recording()
        
        # Calculate final duration
        if st.session_state.start_time:
            final_duration = time.time() - st.session_state.start_time
            st.success(f"Recording stopped! Duration: {final_duration:.2f} seconds")
        
        st.session_state.recording = False
        st.session_state.start_time = None
        st.rerun()

# Status indicator
if st.session_state.recording:
    st.warning("🔴 Recording in progress...")
else:
    st.info("⚪ Ready to record")

